In [68]:
import pandas as pd

df = pd.read_csv(r"Hotels_data_Changed.csv")
df.columns
df.head(5)

,Snapshot ID,Snapshot Date,Checkin Date,Days,Original Price,Discount Price,Discount Code,Available Rooms,Hotel Name,Hotel Stars,WeekDay,DiscountDiff,DiscountPerc,DayDiff
0,1,2015-07-17,2015-08-12,5,1178,1040,1,6,Best Western Plus Seaport Inn Downtown,3,Wed,138,11.714771,26
1,1,2015-07-17,2015-08-19,5,1113,982,1,8,Best Western Plus Seaport Inn Downtown,3,Wed,131,11.769991,33
2,1,2015-07-17,2015-08-13,5,4370,4240,1,3,The Peninsula New York,5,Thu,130,2.974828,27
3,1,2015-07-17,2015-07-26,5,1739,1667,1,18,Eventi Hotel a Kimpton Hotel,4,Sun,72,4.140311,9
4,1,2015-07-17,2015-08-12,5,1739,1672,1,3,Eventi Hotel a Kimpton Hotel,4,Wed,67,3.852789,26


In [69]:
relevantColumns = ['Hotel Name', 'DayDiff', 'Checkin Date', 'Snapshot Date', 'WeekDay', 'Discount Code','DiscountPerc']
relevantColumnsFeatures = relevantColumns[0:6]

df = df[relevantColumns]
df.sort_values(['DiscountPerc'], ascending=False, inplace=True)
df.drop_duplicates(subset=relevantColumnsFeatures, keep='first', inplace=True)
df.dropna()
df.to_csv(r"Hotels_data_filtered.csv")
df.head(5)

,Hotel Name,DayDiff,Checkin Date,Snapshot Date,WeekDay,Discount Code,DiscountPerc
26262,Hyatt Place Flushing LaGuardia Airport,1,2015-09-03,2015-09-02,Thu,1,68.425842
69313,Manhattan Broadway Budget Hotel,21,2015-10-22,2015-10-01,Thu,3,68.305669
68905,Manhattan Broadway Budget Hotel,19,2015-10-20,2015-10-01,Tue,2,66.615761
146691,Comfort Inn Times Square West,5,2015-12-01,2015-11-26,Tue,3,56.410256
146304,Comfort Inn Times Square West,5,2015-12-01,2015-11-26,Tue,2,56.410256


In [70]:
#Convert the data to numeric
from sklearn import preprocessing

#Hotel Name
le = preprocessing.LabelEncoder()
le.fit(df['Hotel Name'])
df['Hotel Name'] = le.transform(df['Hotel Name']) 

#WeekDay
le = preprocessing.LabelEncoder()
le.fit(df['WeekDay'])
df['WeekDay'] = le.transform(df['WeekDay']) 

#Snapshot Date
le = preprocessing.LabelEncoder()
le.fit(df['Snapshot Date'])
df['Snapshot Date'] = le.transform(df['Snapshot Date']) 

#Checkin Date
le = preprocessing.LabelEncoder()
le.fit(df['Checkin Date'])
df['Checkin Date'] = le.transform(df['Checkin Date']) 

In [101]:
# Naive Bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

features = df[relevantColumnsFeatures]
target = df['Discount Code'].values

f_train, f_test, t_train, t_test = train_test_split(features, target,test_size=0.33, random_state=42)

nb = GaussianNB()

nb.fit(f_train, t_train)
predicted = nb.predict(f_test)
predicted_probas = nb.predict_proba(f_test)
print(accuracy_score(t_test, predicted))

1.0


In [108]:
#for test!!!
df4= f_test.iloc[0:2]
df4['DayDiff'] = 5
print(df4)
predicted2 = nb.predict(df4)
predicted_probas2 = nb.predict_proba(df4)
print(predicted2)

        Hotel Name  DayDiff  Checkin Date  Snapshot Date  WeekDay  \
124156         428        5           112            107        2   
174386         436        5           183            148        3   

        Discount Code  
124156              2  
174386              3  
[2 3]


/Users/eli.l/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [85]:
from sklearn.metrics import confusion_matrix
import numpy as np

matrix=pd.DataFrame(
    confusion_matrix(t_test, predicted),
    columns=['Predicted 1', 'Predicted 2','Predicted 3','Predicted 4'],
    index=['True 1', 'True 2','True 3','True 4']
)

print("-------------------------STATISTICS-------------------------")
print("confusion_matrix:")
print(matrix)
print("------------------------------------------------------------")
# accuracy
accuracy=accuracy_score(t_test, predicted)
print("accuracy is: %s" %(accuracy))
print("------------------------------------------------------------")
# TP
tp = np.diag(matrix)
print("TP is: %s" %(tp))
print("------------------------------------------------------------")
# FP
fp=matrix.sum(axis=0)-np.diag(matrix)
print("FP:")
print(fp)
print("------------------------------------------------------------")
# FN
fn = matrix.sum(axis=1) - np.diag(matrix)
print("FN:")
print(fn)
print("------------------------------------------------------------")
# ROC
print("ROC:")
# This is the ROC curve
skplt.metrics.plot_roc_curve(y1_test, predicted_probas)
plt.show()
print("see diagram")
print("------------------------------------------------------------")
print()

#Eli End! ---------------------

-------------------------STATISTICS-------------------------
confusion_matrix:
        Predicted 1  Predicted 2  Predicted 3  Predicted 4
True 1         9724            0            0            0
True 2            0        14889            0            0
True 3            0            0        13816            0
True 4            0            0            0         8002
------------------------------------------------------------
accuracy is: 1.0
------------------------------------------------------------
TP is: [ 9724 14889 13816  8002]
------------------------------------------------------------
FP:
Predicted 1    0
Predicted 2    0
Predicted 3    0
Predicted 4    0
dtype: int64
------------------------------------------------------------
FN:
True 1    0
True 2    0
True 3    0
True 4    0
dtype: int64
------------------------------------------------------------
ROC:


NameError: name 'skplt' is not defined

In [37]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

train, test, train_labels, test_labels = train_test_split(counts,
                                                          targets,
                                                          test_size=0.33,
                                                          random_state=42)
# Initialize our classifier
gnb = GaussianNB()

# Train our classifier
model = gnb.fit(train.toarray(), train_labels)


GaussianNB(priors=None)


In [36]:

# Make predictions
preds = gnb.predict(test)
print(preds)

# Evaluate accuracy
print(accuracy_score(test_labels, preds))


TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

In [32]:
# Decision tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder


data_for_tree = df.copy()
del data_for_tree['Snapshot ID']
del data_for_tree['Days']
del data_for_tree['Original Price']
del data_for_tree['Discount Price']
del data_for_tree['Available Rooms']
del data_for_tree['Hotel Stars']
del data_for_tree['DiscountDiff']

le = LabelEncoder()

le_weekday = LabelEncoder()
le_weekday.fit(data_for_tree['WeekDay'].unique())
data_for_tree['WeekDay'] = le_weekday.transform(data_for_tree['WeekDay'].values)

le_hotel = LabelEncoder()
le_hotel.fit(data_for_tree['Hotel Name'].unique())
data_for_tree['Hotel Name'] = le_hotel.transform(data_for_tree['Hotel Name'].values)

le_ci_date = LabelEncoder()
le_ci_date.fit(data_for_tree['Checkin Date'].unique())
data_for_tree['Checkin Date'] = le_ci_date.transform(data_for_tree['Checkin Date'].values)

le_ss_date = LabelEncoder()
le_ss_date.fit(data_for_tree['Snapshot Date'].unique())
data_for_tree['Snapshot Date'] = le_ss_date.transform(data_for_tree['Snapshot Date'].values)

X = data_for_tree.values
Y = data_for_tree.values

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 100)


dt = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=100, splitter='best')
dt.fit(X_train,y_train)


ValueError: Unknown label type: 'continuous-multioutput'